In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

edu_df = pd.read_sql_query('select * from useducation', con=engine)
engine.dispose()

In [32]:
edu_df.info()
edu_df.head(60)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PRIMARY_KEY                   1492 non-null   object 
 1   STATE                         1492 non-null   object 
 2   YEAR                          1492 non-null   int64  
 3   ENROLL                        1229 non-null   float64
 4   TOTAL_REVENUE                 1280 non-null   float64
 5   FEDERAL_REVENUE               1280 non-null   float64
 6   STATE_REVENUE                 1280 non-null   float64
 7   LOCAL_REVENUE                 1280 non-null   float64
 8   TOTAL_EXPENDITURE             1280 non-null   float64
 9   INSTRUCTION_EXPENDITURE       1280 non-null   float64
 10  SUPPORT_SERVICES_EXPENDITURE  1280 non-null   float64
 11  OTHER_EXPENDITURE             1229 non-null   float64
 12  CAPITAL_OUTLAY_EXPENDITURE    1280 non-null   float64
 13  GRA

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN
5,1992_COLORADO,COLORADO,1992,NaN,3185173.0,163253.0,1307986.0,1713934.0,3264826.0,1642466.0,...,50648.0,45025.0,34533.0,394904.0,160299.0,562613.0,221.023429,272.398433,213.480890,266.419687
6,1992_CONNECTICUT,CONNECTICUT,1992,NaN,3834302.0,143542.0,1342539.0,2348221.0,3721338.0,2148041.0,...,38058.0,33691.0,28366.0,304284.0,126917.0,436932.0,226.798480,273.739345,222.417599,264.713904
7,1992_DELAWARE,DELAWARE,1992,NaN,645233.0,45945.0,420942.0,178346.0,638784.0,372722.0,...,8272.0,8012.0,6129.0,67495.0,28338.0,96296.0,217.899972,262.868585,206.389052,261.577921
8,1992_DISTRICT_OF_COLUMBIA,DISTRICT_OF_COLUMBIA,1992,NaN,709480.0,64749.0,0.0,644731.0,742893.0,329160.0,...,5832.0,5000.0,3433.0,47009.0,18173.0,70000.0,192.600553,234.924621,178.557612,265.976765
9,1992_FLORIDA,FLORIDA,1992,NaN,11506299.0,788420.0,5683949.0,5033930.0,11305642.0,5166374.0,...,164416.0,142372.0,100835.0,1276685.0,511557.0,1819706.0,213.689751,259.908848,204.882606,262.118986


In [33]:
# Step 1 of EDA: Data cleaning.
# We need to fill the null values. 

edu_df = edu_df.sort_values(by=["PRIMARY_KEY"])

for column in edu_df.columns:
    edu_df.loc[:, column] = edu_df.loc[:, column].interpolate()
    
# We still have a few missing values in "ENROLL" and "OTHER_EXPENDITURE",
# so we will just drop those rows.

edu_df = edu_df.dropna(inplace=False)

edu_df.info()
edu_df.head(60)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1436 entries, 52 to 1491
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PRIMARY_KEY                   1436 non-null   object 
 1   STATE                         1436 non-null   object 
 2   YEAR                          1436 non-null   int64  
 3   ENROLL                        1436 non-null   float64
 4   TOTAL_REVENUE                 1436 non-null   float64
 5   FEDERAL_REVENUE               1436 non-null   float64
 6   STATE_REVENUE                 1436 non-null   float64
 7   LOCAL_REVENUE                 1436 non-null   float64
 8   TOTAL_EXPENDITURE             1436 non-null   float64
 9   INSTRUCTION_EXPENDITURE       1436 non-null   float64
 10  SUPPORT_SERVICES_EXPENDITURE  1436 non-null   float64
 11  OTHER_EXPENDITURE             1436 non-null   float64
 12  CAPITAL_OUTLAY_EXPENDITURE    1436 non-null   float64
 13  GR

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
52,1993_ALABAMA,ALABAMA,1993,727716.0,2827391.0,331409.0,1729295.0,766687.0,2833433.0,1564558.0,...,57497.0,60004.0,39900.0,470775.0,198651.0,677690.0,225.296345,274.970696,221.124817,261.729716
53,1993_ALASKA,ALASKA,1993,121156.0,1191398.0,176150.0,775829.0,239419.0,1126398.0,494917.0,...,10156.0,9160.0,6975.0,80485.0,32347.0,115619.0,225.215099,274.861316,221.064079,261.721066
1400,1993_AMERICAN_SAMOA,AMERICAN_SAMOA,1993,398726.5,2309687.0,247307.5,1095618.0,966761.5,2375172.0,1036903.0,...,1056.0,873.0,809.0,8255.0,3451.0,13369.0,225.133853,274.751936,221.003341,261.712415
54,1993_ARIZONA,ARIZONA,1993,676297.0,3427976.0,318465.0,1415407.0,1694104.0,3623946.0,1578889.0,...,57701.0,53500.0,37614.0,461398.0,182737.0,647299.0,225.052607,274.642556,220.942602,261.703765
55,1993_ARKANSAS,ARKANSAS,1993,311432.0,1346909.0,128196.0,771079.0,447634.0,1376067.0,782791.0,...,34255.0,36471.0,27169.0,280280.0,125801.0,407329.0,224.971360,274.533176,220.881864,261.695114
56,1993_CALIFORNIA,CALIFORNIA,1993,5129788.0,28043338.0,2151157.0,17064146.0,8828035.0,28110986.0,15281147.0,...,420233.0,380223.0,277271.0,3328627.0,1393530.0,4782111.0,224.890114,274.423796,220.821126,261.686464
57,1993_COLORADO,COLORADO,1993,539538.0,3058326.0,147793.0,1242337.0,1668196.0,3028305.0,1537714.0,...,50807.0,47665.0,34770.0,403871.0,164260.0,575380.0,224.808868,274.314416,220.760387,261.677813
58,1993_CONNECTICUT,CONNECTICUT,1993,471918.0,4064158.0,149204.0,1433539.0,2481415.0,4079943.0,2302852.0,...,39093.0,34662.0,27979.0,310022.0,127655.0,443893.0,224.727622,274.205036,220.699649,261.669163
59,1993_DELAWARE,DELAWARE,1993,104355.0,683954.0,44497.0,443250.0,196207.0,694534.0,394680.0,...,8285.0,8265.0,5882.0,67899.0,28930.0,97394.0,224.646375,274.095656,220.638910,261.660513
60,1993_DISTRICT_OF_COLUMBIA,DISTRICT_OF_COLUMBIA,1993,80937.0,721151.0,73882.0,0.0,647269.0,719440.0,339266.0,...,5870.0,4935.0,3303.0,46917.0,17854.0,69987.0,224.565129,273.986276,220.578172,261.651862


# 1. Compute the average score for each row for all students weighted properly by grade. In other words, each row will have its own average. Notice that the number of students in the fourth grade isn't the same as the number of students in the eighth grade. So, you should appropriately weigh the scores.

In [39]:
edu_df["WEIGHTED_AVG"] = (
    ((edu_df["GRADES_4_G"] * 
    ((edu_df["AVG_MATH_4_SCORE"] + edu_df["AVG_READING_4_SCORE"]) / 2)) +
    (edu_df["GRADES_8_G"] *
    ((edu_df["AVG_MATH_8_SCORE"] + edu_df["AVG_READING_8_SCORE"]) / 2))) /
    (edu_df["GRADES_4_G"] + edu_df["GRADES_8_G"])
)

edu_df.info()
edu_df.head(60)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1436 entries, 52 to 1491
Data columns (total 26 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PRIMARY_KEY                   1436 non-null   object 
 1   STATE                         1436 non-null   object 
 2   YEAR                          1436 non-null   int64  
 3   ENROLL                        1436 non-null   float64
 4   TOTAL_REVENUE                 1436 non-null   float64
 5   FEDERAL_REVENUE               1436 non-null   float64
 6   STATE_REVENUE                 1436 non-null   float64
 7   LOCAL_REVENUE                 1436 non-null   float64
 8   TOTAL_EXPENDITURE             1436 non-null   float64
 9   INSTRUCTION_EXPENDITURE       1436 non-null   float64
 10  SUPPORT_SERVICES_EXPENDITURE  1436 non-null   float64
 11  OTHER_EXPENDITURE             1436 non-null   float64
 12  CAPITAL_OUTLAY_EXPENDITURE    1436 non-null   float64
 13  GR

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,WEIGHTED_AVG
52,1993_ALABAMA,ALABAMA,1993,727716.0,2827391.0,331409.0,1729295.0,766687.0,2833433.0,1564558.0,...,60004.0,39900.0,470775.0,198651.0,677690.0,225.296345,274.970696,221.124817,261.729716,246.261943
53,1993_ALASKA,ALASKA,1993,121156.0,1191398.0,176150.0,775829.0,239419.0,1126398.0,494917.0,...,9160.0,6975.0,80485.0,32347.0,115619.0,225.215099,274.861316,221.064079,261.721066,244.551303
1400,1993_AMERICAN_SAMOA,AMERICAN_SAMOA,1993,398726.5,2309687.0,247307.5,1095618.0,966761.5,2375172.0,1036903.0,...,873.0,809.0,8255.0,3451.0,13369.0,225.133853,274.751936,221.003341,261.712415,243.508101
54,1993_ARIZONA,ARIZONA,1993,676297.0,3427976.0,318465.0,1415407.0,1694104.0,3623946.0,1578889.0,...,53500.0,37614.0,461398.0,182737.0,647299.0,225.052607,274.642556,220.942602,261.703765,244.732052
55,1993_ARKANSAS,ARKANSAS,1993,311432.0,1346909.0,128196.0,771079.0,447634.0,1376067.0,782791.0,...,36471.0,27169.0,280280.0,125801.0,407329.0,224.971360,274.533176,220.881864,261.695114,246.228291
56,1993_CALIFORNIA,CALIFORNIA,1993,5129788.0,28043338.0,2151157.0,17064146.0,8828035.0,28110986.0,15281147.0,...,380223.0,277271.0,3328627.0,1393530.0,4782111.0,224.890114,274.423796,220.821126,261.686464,244.325749
57,1993_COLORADO,COLORADO,1993,539538.0,3058326.0,147793.0,1242337.0,1668196.0,3028305.0,1537714.0,...,47665.0,34770.0,403871.0,164260.0,575380.0,224.808868,274.314416,220.760387,261.677813,244.669077
58,1993_CONNECTICUT,CONNECTICUT,1993,471918.0,4064158.0,149204.0,1433539.0,2481415.0,4079943.0,2302852.0,...,34662.0,27979.0,310022.0,127655.0,443893.0,224.727622,274.205036,220.699649,261.669163,243.966916
59,1993_DELAWARE,DELAWARE,1993,104355.0,683954.0,44497.0,443250.0,196207.0,694534.0,394680.0,...,8265.0,5882.0,67899.0,28930.0,97394.0,224.646375,274.095656,220.638910,261.660513,245.233031
60,1993_DISTRICT_OF_COLUMBIA,DISTRICT_OF_COLUMBIA,1993,80937.0,721151.0,73882.0,0.0,647269.0,719440.0,339266.0,...,4935.0,3303.0,46917.0,17854.0,69987.0,224.565129,273.986276,220.578172,261.651862,243.237640


# 2. What are the correlations between this newly created score variable and the expenditure types? Which one of the expenditure types is more correlated than the others?

In [46]:
w_avg_expend = edu_df[["WEIGHTED_AVG", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE", 
                       "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE",
                       "CAPITAL_OUTLAY_EXPENDITURE"]]

w_avg_expend.corr()

,WEIGHTED_AVG,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
WEIGHTED_AVG,1.000000,0.142128,0.145810,0.154596,0.100505,0.073002
TOTAL_EXPENDITURE,0.142128,1.000000,0.991077,0.993312,0.946471,0.930095
INSTRUCTION_EXPENDITURE,0.145810,0.991077,1.000000,0.976078,0.908616,0.892877
SUPPORT_SERVICES_EXPENDITURE,0.154596,0.993312,0.976078,1.000000,0.954775,0.916469
OTHER_EXPENDITURE,0.100505,0.946471,0.908616,0.954775,1.000000,0.921591
CAPITAL_OUTLAY_EXPENDITURE,0.073002,0.930095,0.892877,0.916469,0.921591,1.000000


### The correlation does not seem significant, although the support services expenditure has the highest correlation (0.15).

# 3. Now, apply PCA to the four expenditure types. How much of the total variance is explained by the first component?

In [90]:
four_expend = edu_df[["SUPPORT_SERVICES_EXPENDITURE", "INSTRUCTION_EXPENDITURE", 
           "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]

X = StandardScaler().fit_transform(four_expend)

sklearn_pca = PCA(n_components=4)
edu_df["PCA_1"] = sklearn_pca.fit_transform(X)

pca = sklearn_pca.explained_variance_ratio_
pca = pd.DataFrame(data=pca)

print(pca)

for col in four_expend.columns:
    for i in pca.iloc[: , 0]:
        print('{} explains the percentage of {} total variance in the dataset according to Sklearn PCA'.format(col, i))
    

          0
0  0.946416
1  0.031069
2  0.019168
3  0.003346
SUPPORT_SERVICES_EXPENDITURE explains the percentage of 0.9464161498808403 total variance in the dataset according to Sklearn PCA
SUPPORT_SERVICES_EXPENDITURE explains the percentage of 0.031069307226513635 total variance in the dataset according to Sklearn PCA
SUPPORT_SERVICES_EXPENDITURE explains the percentage of 0.0191684204061436 total variance in the dataset according to Sklearn PCA
SUPPORT_SERVICES_EXPENDITURE explains the percentage of 0.0033461224865023826 total variance in the dataset according to Sklearn PCA
INSTRUCTION_EXPENDITURE explains the percentage of 0.9464161498808403 total variance in the dataset according to Sklearn PCA
INSTRUCTION_EXPENDITURE explains the percentage of 0.031069307226513635 total variance in the dataset according to Sklearn PCA
INSTRUCTION_EXPENDITURE explains the percentage of 0.0191684204061436 total variance in the dataset according to Sklearn PCA
INSTRUCTION_EXPENDITURE explains the p

# 4. What is the correlation between the overall score variable and the first principal component?

In [89]:
edu_df[["WEIGHTED_AVG", "PCA_1"]].corr()

,WEIGHTED_AVG,PCA_1
WEIGHTED_AVG,1.000000,0.122093
PCA_1,0.122093,1.000000


# 5. If you were to choose the best variables for your model, would you prefer using the first principal component instead of the expenditure variables? Why?

### No. The support services expenditure variable has the highest correlation. Thus, I'd use the support services expenditure. 

### The answer sheet says: instruction expenditure variable is more correlated with the overall score than the first principal component. Hence using instruciton expenditure makes more sense. PCA works best when the correlation between the variables are less than or equal to 0.8. In our case, all of the expenditure vairabales are highly correlated with each ther. this will result in instable principal component estimations. 